In [1]:
import sys

sys.path.append('..')

from dataset import Pipeline, B, V, C, Config
from dataset.opensets import MNIST
from dataset.models.tf import VGG7
from multiplerun import SingleRunning, MultipleRunning
from distributor import Tasks, PipelineWorker

In [2]:
placeholders_config = {
    'images': {'shape': (28, 28, 1),
               'type': 'float32',
               'name': 'reshaped_images'
              },
    'labels': {'classes': 10,
               'type': 'int32',
               'transform': 'ohe',
               'name': 'targets'
              }
}

feed_dict = {'images': B('images'),
             'labels': B('labels')}

In [3]:
config = {'inputs': placeholders_config,
          'input_block/inputs': 'images',
          'batch_norm': {'momentum': 0.1},
          'output': {'ops': ['accuracy']},
          'loss': 'ce',
          'optimizer': 'Adam'}

ppl_config = {'model_config': config}

In [4]:
mnist = MNIST()
train_ppl = (mnist.train.p
             .init_variable('loss', init_on_each_run=list)
             .init_model('dynamic', VGG7, 'model', C('model_config'))
             .train_model('model', feed_dict=feed_dict, fetches='loss', save_to=V('loss'), mode='a')
             .run(batch_size=10, shuffle=True, n_epochs=None, lazy=True)
            )

test_ppl = (mnist.train.p
             .init_variable('accuracy', init_on_each_run=list)
             .import_model('model', train_ppl)
             .predict_model('model', feed_dict=feed_dict, fetches='output_accuracy', save_to=V('accuracy'), mode='a')
             .run(batch_size=32, shuffle=True, n_epochs=None, lazy=True)
            )

ExtractingExtractingExtractingExtracting    C:\Users\kozhevin\AppData\Local\Temp\t10k-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\t10k-images-idx3-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\train-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\train-images-idx3-ubyte.gz





In [5]:
mr = MultipleRunning()
mr.add_pipeline(train_ppl, 'loss', Config(ppl_config))
mr.add_pipeline(test_ppl, 'accuracy')
mr.add_

In [6]:
mr.pipelines

[{'cfg': Config({'model_config': {'loss': 'ce', 'output': {'ops': ['accuracy']}, 'optimizer': 'Adam', 'input_block': {'inputs': 'images'}, 'inputs': {'labels': {'name': 'targets', 'transform': 'ohe', 'classes': 10, 'type': 'int32'}, 'images': {'name': 'reshaped_images', 'shape': (28, 28, 1), 'type': 'float32'}}, 'batch_norm': {'momentum': 0.1}}}),
  'name': 'ppl_0',
  'ppl': <dataset.dataset.pipeline.Pipeline at 0x1d2e424cb00>,
  'var': ['loss']},
 {'cfg': Config({}),
  'name': 'ppl_1',
  'ppl': <dataset.dataset.pipeline.Pipeline at 0x1d2e424cf60>,
  'var': ['accuracy']}]

In [7]:
res = mr.run(10, 10)

AttributeError: 'MultipleRunning' object has no attribute 'grid_config'

In [11]:
res.stat

{'ppl_0': {'loss': [[2.4249415,
    139.95316,
    368.50092,
    461.90274,
    262.2561,
    208.83847,
    250.88113,
    265.85931,
    134.89742,
    107.90481]]},
 'ppl_1': {'accuracy': [[0.125,
    0.0625,
    0.1875,
    0.125,
    0.0625,
    0.0625,
    0.1875,
    0.125,
    0.03125,
    0.15625]]}}

In [13]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep
 
def return_after_5_secs(message):
    sleep(5)
    return message
 
pool = ThreadPoolExecutor(3)
 
future = pool.submit(return_after_5_secs, ("hello"))
print(future.done())
sleep(5)
print(future.done())
print(future.result())

False
False
hello


In [14]:
print(future.done())

True
